Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [2]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle


hidden_size = 1024
batch_size = 128
image_size = 28
num_labels = 10

lambda_reg = 5e-4

First reload the data we generated in _notmist.ipynb_.

In [28]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    train_dataset = save['train_dataset']
    train_labels = save['train_labels']
    valid_dataset = save['valid_dataset']
    valid_labels = save['valid_labels']
    test_dataset = save['test_dataset']
    test_labels = save['test_labels']
    
    del save  # hint to help gc free up memory
    
    print('Training set', train_dataset.shape, train_labels.shape)
    print('Validation set', valid_dataset.shape, valid_labels.shape)
    print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [29]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
    dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
    labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
    return dataset, labels

train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)

print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [27]:
def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1)) / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [6]:
# Model graph definition
graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    in2h_weights = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_size]))
    h_biases = tf.Variable(tf.zeros([hidden_size]))
    h2out_weights = tf.Variable(tf.truncated_normal([hidden_size, num_labels]))
    out_biases = tf.Variable(tf.zeros([num_labels]))

    # Training computation graph.
    h_activations = tf.nn.relu(tf.matmul(tf_train_dataset, in2h_weights) + h_biases)
    logits = tf.matmul(h_activations, h2out_weights) + out_biases
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
    # regularization
    reg_terms = tf.nn.l2_loss(in2h_weights) + tf.nn.l2_loss(h2out_weights) + \
                tf.nn.l2_loss(h_biases) + tf.nn.l2_loss(out_biases)
        
    loss += lambda_reg * reg_terms

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.2).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)


    valid_prediction = tf.nn.softmax(
                            tf.matmul(
                                tf.nn.relu(tf.matmul(tf_valid_dataset, in2h_weights) + h_biases),
                                h2out_weights
                                )
                        + out_biases)


    test_prediction = tf.nn.softmax(
                            tf.matmul(
                                tf.nn.relu(tf.matmul(tf_test_dataset, in2h_weights) + h_biases),
                                h2out_weights
                                )
                        + out_biases)
    
    
    
# Model training (execution)
num_steps = 10000

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")

    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}

        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        
        if (step % 500 == 0):
            print("\nMinibatch loss at step %d: %f" % (step, l))
            print(" > Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print(" > Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))

    print("\n")
    print("-"*40)
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized

Minibatch loss at step 0: 690.469055
 > Minibatch accuracy: 1.6%
 > Validation accuracy: 34.5%

Minibatch loss at step 500: 154.552170
 > Minibatch accuracy: 82.0%
 > Validation accuracy: 78.5%

Minibatch loss at step 1000: 134.413971
 > Minibatch accuracy: 85.9%
 > Validation accuracy: 83.5%

Minibatch loss at step 1500: 124.096046
 > Minibatch accuracy: 82.8%
 > Validation accuracy: 81.1%

Minibatch loss at step 2000: 108.881355
 > Minibatch accuracy: 76.6%
 > Validation accuracy: 83.7%

Minibatch loss at step 2500: 95.429344
 > Minibatch accuracy: 84.4%
 > Validation accuracy: 81.8%

Minibatch loss at step 3000: 85.954475
 > Minibatch accuracy: 85.9%
 > Validation accuracy: 83.9%

Minibatch loss at step 3500: 77.230011
 > Minibatch accuracy: 85.9%
 > Validation accuracy: 84.7%

Minibatch loss at step 4000: 69.502724
 > Minibatch accuracy: 87.5%
 > Validation accuracy: 83.9%

Minibatch loss at step 4500: 63.821678
 > Minibatch accuracy: 86.7%
 > Validation accuracy: 83.9

---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [7]:
# Model training (execution)
train_subset = 1000
num_steps = 2500

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    
    # Use only a portion of the data --> force overfitting
    print(train_dataset.shape)
    train_data_subset = train_dataset[:train_subset, :]
    train_labels_subset = train_labels[:train_subset, :]
    print(train_data_subset.shape)

    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels_subset.shape[0] - batch_size)
        
        # Generate a minibatch.
        batch_data = train_data_subset[offset:(offset + batch_size), :]
        batch_labels = train_labels_subset[offset:(offset + batch_size), :]
        
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}

        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        
        if (step % 500 == 0):
            print("\nMinibatch loss at step %d: %f" % (step, l))
            print(" > Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print(" > Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))

    print("\n")
    print("-"*40)
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
(200000, 784)
(1000, 784)

Minibatch loss at step 0: 536.934204
 > Minibatch accuracy: 10.9%
 > Validation accuracy: 34.9%

Minibatch loss at step 500: 142.204010
 > Minibatch accuracy: 100.0%
 > Validation accuracy: 76.8%

Minibatch loss at step 1000: 128.670517
 > Minibatch accuracy: 100.0%
 > Validation accuracy: 76.7%

Minibatch loss at step 1500: 116.426414
 > Minibatch accuracy: 100.0%
 > Validation accuracy: 76.5%

Minibatch loss at step 2000: 105.346695
 > Minibatch accuracy: 100.0%
 > Validation accuracy: 76.6%


----------------------------------------
Test accuracy: 82.9%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [19]:
# Model graph definition (now with dropout)

lambda_reg = 5e-2

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    in2h_weights = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_size]))
    h_biases = tf.Variable(tf.zeros([hidden_size]))
    h2out_weights = tf.Variable(tf.truncated_normal([hidden_size, num_labels]))
    out_biases = tf.Variable(tf.zeros([num_labels]))

    # Training computation graph.
    h_activations = tf.nn.relu(tf.matmul(tf_train_dataset, in2h_weights) + h_biases)
    drop_h_activations = tf.nn.dropout(h_activations, 0.3)
    logits = tf.matmul(drop_h_activations, h2out_weights) + out_biases
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
    # regularization
    reg_terms = tf.nn.l2_loss(in2h_weights) + tf.nn.l2_loss(h2out_weights) + \
                tf.nn.l2_loss(h_biases) + tf.nn.l2_loss(out_biases)
        
    loss += lambda_reg * reg_terms

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.2).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)

    def pred(dataset):
        return tf.nn.softmax(
                            tf.matmul(
                                tf.nn.relu(tf.matmul(dataset, in2h_weights) + h_biases),
                                h2out_weights
                                )
                        + out_biases)
    
    # graphs for validation and test (no dropout, no loss)
    valid_prediction = pred(tf_valid_dataset)
    test_prediction = pred(tf_test_dataset)

    
    
# Model training (execution)
train_subset = 1000
num_steps = 2500

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    
    # Use only a portion of the data --> force overfitting
    print(train_dataset.shape)
    train_data_subset = train_dataset[:train_subset, :]
    train_labels_subset = train_labels[:train_subset, :]
    print(train_data_subset.shape)

    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels_subset.shape[0] - batch_size)
        
        # Generate a minibatch.
        batch_data = train_data_subset[offset:(offset + batch_size), :]
        batch_labels = train_labels_subset[offset:(offset + batch_size), :]
        
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}

        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        
        if (step % 500 == 0):
            print("\nMinibatch loss at step %d: %f" % (step, l))
            print(" > Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print(" > Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))

    print("\n")
    print("-"*40)
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
(200000, 784)
(1000, 784)

Minibatch loss at step 0: 16247.412109
 > Minibatch accuracy: 14.8%
 > Validation accuracy: 38.4%

Minibatch loss at step 500: 1.763888
 > Minibatch accuracy: 87.5%
 > Validation accuracy: 80.3%

Minibatch loss at step 1000: 1.135774
 > Minibatch accuracy: 86.7%
 > Validation accuracy: 80.5%

Minibatch loss at step 1500: 1.139773
 > Minibatch accuracy: 82.0%
 > Validation accuracy: 80.6%

Minibatch loss at step 2000: 1.087514
 > Minibatch accuracy: 85.9%
 > Validation accuracy: 80.2%


----------------------------------------
Test accuracy: 87.0%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [37]:
# Model graph definition (now with dropout)

lambda_reg = 5e-4

decay_rate = 0.96
decay_steps = 25000
starter_learning_rate = 0.5

hidden_size_1 = 1024
hidden_size_2 = 300
hidden_size_3 = 50


graph = tf.Graph()
with graph.as_default():
    
    # Count number of steps taken
    global_step = tf.Variable(0, trainable=False)

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    # hidden layer 1
    h1_weights = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_size_1]))
    h1_biases = tf.Variable(tf.zeros([hidden_size_1]))
    
    # hidden 2
    h2_weights = tf.Variable(tf.truncated_normal([hidden_size_1, hidden_size_2]))
    h2_biases = tf.Variable(tf.zeros([hidden_size_2]))
    
    # hidden 3
    h3_weights = tf.Variable(tf.truncated_normal([hidden_size_2, hidden_size_3]))
    h3_biases = tf.Variable(tf.zeros([hidden_size_3]))
    
    # hidden to output
    h2out_weights = tf.Variable(tf.truncated_normal([hidden_size_3, num_labels]))
    out_biases = tf.Variable(tf.zeros([num_labels]))

    # Training computation graph.
    h1_activations = tf.nn.tanh(tf.matmul(tf_train_dataset, h1_weights) + h1_biases)
    drop_h1_activations = tf.nn.dropout(h1_activations, 0.8)
    h2_activations = tf.nn.tanh(tf.matmul(drop_h1_activations, h2_weights) + h2_biases)
    drop_h2_activations = tf.nn.dropout(h2_activations, 0.7)
    h3_activations = tf.nn.tanh(tf.matmul(drop_h2_activations, h3_weights) + h3_biases)
    drop_h3_activations = tf.nn.dropout(h3_activations, 0.6)
    logits = tf.matmul(drop_h3_activations, h2out_weights) + out_biases
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
    # regularization
    reg_terms = tf.nn.l2_loss(h1_weights) + tf.nn.l2_loss(h1_biases) + \
                tf.nn.l2_loss(h2_weights) + tf.nn.l2_loss(h2_biases) + \
                tf.nn.l2_loss(h3_weights) + tf.nn.l2_loss(h3_biases)

        
    loss += lambda_reg * reg_terms

    # Optimizer.
    learning_rate = tf.train.exponential_decay(starter_learning_rate, 
                                               global_step, 
                                               decay_steps, 
                                               decay_rate, 
                                               staircase=True)
    
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)

    def pred(dataset):
        h1_activations = tf.nn.tanh(tf.matmul(dataset, h1_weights) + h1_biases)
        h2_activations = tf.nn.tanh(tf.matmul(h1_activations, h2_weights) + h2_biases)
        h3_activations = tf.nn.tanh(tf.matmul(h2_activations, h3_weights) + h3_biases)
        logits = tf.matmul(h3_activations, h2out_weights) + out_biases
        return tf.nn.softmax(logits)
    
    # graphs for validation and test (no dropout, no loss)
    valid_prediction = pred(tf_valid_dataset)
    test_prediction = pred(tf_test_dataset)

    
    
# Model training (execution)
num_steps = 35000

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels_subset.shape[0] - batch_size)
        
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}

        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        
        if (step % 500 == 0):
            print("\nMinibatch loss at step %d: %f" % (step, l))
            print(" > Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print(" > Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))

    print("\n")
    print("-"*40)
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))
    
 

Initialized

Minibatch loss at step 0: 11.790779
 > Minibatch accuracy: 12.5%
 > Validation accuracy: 11.2%

Minibatch loss at step 500: 1.170437
 > Minibatch accuracy: 62.5%
 > Validation accuracy: 65.0%

Minibatch loss at step 1000: 1.279820
 > Minibatch accuracy: 57.0%
 > Validation accuracy: 70.3%

Minibatch loss at step 1500: 1.206752
 > Minibatch accuracy: 63.3%
 > Validation accuracy: 73.3%

Minibatch loss at step 2000: 0.791922
 > Minibatch accuracy: 73.4%
 > Validation accuracy: 73.9%

Minibatch loss at step 2500: 0.851385
 > Minibatch accuracy: 71.1%
 > Validation accuracy: 74.9%

Minibatch loss at step 3000: 0.635985
 > Minibatch accuracy: 80.5%
 > Validation accuracy: 76.0%

Minibatch loss at step 3500: 0.672033
 > Minibatch accuracy: 76.6%
 > Validation accuracy: 76.3%

Minibatch loss at step 4000: 0.572926
 > Minibatch accuracy: 82.8%
 > Validation accuracy: 76.6%

Minibatch loss at step 4500: 0.377501
 > Minibatch accuracy: 87.5%
 > Validation accuracy: 76.7%

Minibatch 